## Installing stuff

In [ ]:
!pip install segmentation-models-pytorch
!pip install torchmetrics
!pip install nnunet
!pip install matplotlib
!pip install opencv-python
!pip install albumentations
!pip install --upgrade google-api-python-client
!pip install google-colab
!apt-get update && apt-get install libgl1 -y

In [ ]:
!nvidia-smi

## Importing and Downloading/Extracting Stuff

In [ ]:
import torch
import torchvision
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

import numpy as np

import matplotlib.pyplot as plt

import pandas as pd

import glob, os

from PIL import Image,ImageOps

from tqdm import tqdm

from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import segmentation_models_pytorch as smp

from torchmetrics import JaccardIndex

import cv2

import albumentations as A



In [ ]:
from zipfile import ZipFile
zf = ZipFile('./Inputs.zip', 'r')
zf.extractall('./Train')
zf.close()

In [ ]:
zf = ZipFile('./test_set.zip', 'r')
zf.extractall('./Test')
zf.close()

## Creating folders

In [ ]:
from pathlib import Path

Path("./nnUNet_raw_data_base").mkdir(parents=True, exist_ok=True)
Path("./nnUNet_preprocessed").mkdir(parents=True, exist_ok=True)
Path("./nnUNet_trained_models").mkdir(parents=True, exist_ok=True)

Path("./nnUNet_raw_data_base/nnUNet_raw_data").mkdir(parents=True, exist_ok=True)

Path("./nnUNet_raw_data_base/nnUNet_raw_data/Task001_DRAC").mkdir(parents=True, exist_ok=True)
Path("./nnUNet_raw_data_base/nnUNet_raw_data/Task001_DRAC/imagesTr").mkdir(parents=True, exist_ok=True)
Path("./nnUNet_raw_data_base/nnUNet_raw_data/Task001_DRAC/imagesTs").mkdir(parents=True, exist_ok=True)
Path("./nnUNet_raw_data_base/nnUNet_raw_data/Task001_DRAC/labelsTr").mkdir(parents=True, exist_ok=True)

Path("./nnUNet_raw_data_base/nnUNet_raw_data/Task002_DRAC").mkdir(parents=True, exist_ok=True)
Path("./nnUNet_raw_data_base/nnUNet_raw_data/Task002_DRAC/imagesTr").mkdir(parents=True, exist_ok=True)
Path("./nnUNet_raw_data_base/nnUNet_raw_data/Task002_DRAC/imagesTs").mkdir(parents=True, exist_ok=True)
Path("./nnUNet_raw_data_base/nnUNet_raw_data/Task002_DRAC/labelsTr").mkdir(parents=True, exist_ok=True)

Path("./nnUNet_raw_data_base/nnUNet_raw_data/Task003_DRAC").mkdir(parents=True, exist_ok=True)
Path("./nnUNet_raw_data_base/nnUNet_raw_data/Task003_DRAC/imagesTr").mkdir(parents=True, exist_ok=True)
Path("./nnUNet_raw_data_base/nnUNet_raw_data/Task003_DRAC/imagesTs").mkdir(parents=True, exist_ok=True)
Path("./nnUNet_raw_data_base/nnUNet_raw_data/Task003_DRAC/labelsTr").mkdir(parents=True, exist_ok=True)

import pathlib
here = pathlib.Path('.').resolve()
nnUNet_raw_data_base = here.joinpath('nnUNet_raw_data_base')
nnUNet_preprocessed = here.joinpath('nnUNet_preprocessed')
results_folder = here.joinpath('results')
os.environ["nnUNet_raw_data_base"] = str(nnUNet_raw_data_base)
os.environ["nnUNet_preprocessed"] = str(nnUNet_preprocessed)
os.environ["RESULTS_FOLDER"] = str(results_folder)

In [ ]:
import numpy as np
from batchgenerators.utilities.file_and_folder_operations import *
from nnunet.dataset_conversion.utils import generate_dataset_json
from nnunet.paths import nnUNet_raw_data, preprocessing_output_dir
from nnunet.utilities.file_conversions import convert_2d_image_to_nifti
imgs = './Train/Inputs/Task1/img/'
masks = './Train/Inputs/Task1/gt/'
root = './nnUNet_raw_data_base/nnUNet_raw_data/Task001_DRAC/imagesTr'
label_root = './nnUNet_raw_data_base/nnUNet_raw_data/Task001_DRAC/labelsTr'


def prepare_data_train(imgs, masks, root, label_root):
    for i in os.listdir(imgs):
        img = os.path.join(imgs, i)
        img_pil = np.array(Image.open(img))
        try:
            img_pil = img_pil[:,:,0]
        except:
            pass
        img_pil = Image.fromarray(img_pil)
        img_pil.save(img)

    for i in os.listdir(imgs):
        img = os.path.join(imgs, i)
        convert_2d_image_to_nifti(img, os.path.join(root, i.split('.')[0]), is_seg=False)
        print('Finish：'+ i )

    print("LABELS")
    for i in os.listdir(masks):
        mask = os.path.join(masks, i)
        img = np.array(Image.open(mask))
        img[img == 255] = 1
        try:
            img = img[:,:,1]
        except:
            pass
        cv2.imwrite(mask, img)

    for i in os.listdir(masks):
        mask = os.path.join(masks, i)
        convert_2d_image_to_nifti(mask, os.path.join(label_root, i.split('.')[0]), is_seg=True,
                                 transform=None)
        print('Finish：'+ i )

In [ ]:
import numpy as np
from batchgenerators.utilities.file_and_folder_operations import *
from nnunet.dataset_conversion.utils import generate_dataset_json
from nnunet.paths import nnUNet_raw_data, preprocessing_output_dir
from nnunet.utilities.file_conversions import convert_2d_image_to_nifti
imgs = './Test/'
root = './nnUNet_raw_data_base/nnUNet_raw_data/Task001_DRAC/imagesTs'

def prepare_data_test(imgs, root):
    for i in os.listdir(imgs):
        img = os.path.join(imgs, i)
        img_pil = np.array(Image.open(img))
        print(img_pil.shape)
        img_pil = img_pil[:,:]
        img_pil = Image.fromarray(img_pil)
        img_pil.save(img)

    for i in os.listdir(imgs):
        img = os.path.join(imgs, i)
        convert_2d_image_to_nifti(img, os.path.join(root, i.split('.')[0]), is_seg=False)
        print('Finish：'+ i )

In [ ]:
def get_json(task):
    
    raw = f"./nnUNet_raw_data_base/nnUNet_raw_data/{task}/"
    generate_dataset_json(os.path.join(raw, 'dataset.json'), os.path.join(raw, 'imagesTr'),  os.path.join(raw, 'imagesTs'), ('G'),
              labels={0: 'background', 1: 'class'}, dataset_name=f'{task}', license='--')


## Task1

In [ ]:
imgs = './Train/Inputs/Task1/img/'
masks = './Train/Inputs/Task1/gt/'
root = './nnUNet_raw_data_base/nnUNet_raw_data/Task001_DRACimagesTr'
label_root = './nnUNet_raw_data_base/nnUNet_raw_data/Task001_DRAC/labelsTr'
prepare_data_train(imgs, masks, root, label_root)

imgs = './Test/'
root = './nnUNet_raw_data_base/nnUNet_raw_data/Task001_DRAC/imagesTs'
prepare_data_test(imgs, root)

get_json("Task001_Eye")

In [ ]:
!nnUNet_plan_and_preprocess -t 001 --verify_dataset_integrity

In [ ]:
!pip show nnunet

## Normal training with CV

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2_DA5 Task001_Eye 0 --npz

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2_DA5 Task001_Eye 1 --npz

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2_DA5 Task001_Eye 2 --npz

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2_DA5 Task001_Eye 3 --npz

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2_DA5 Task001_Eye 4 --npz

In [ ]:
!nnUNet_find_best_configuration -tr nnUNetTrainerV2_DA5 -m 2d -t 001 

In [ ]:
!nnUNet_predict -i nnUNet_raw_data_base/nnUNet_raw_data/Task001_Eye/imagesTs/ -o ./output/ -tr nnUNetTrainerV2_DA5 -ctr nnUNetTrainerV2CascadeFullRes -m 2d -p nnUNetPlansv2.1 -t Task001_Eye

## Task2

In [ ]:
imgs = './Train/Inputs/Task2/img/'
masks = './Train/Inputs/Task2/gt/'
root = './nnUNet_raw_data_base/nnUNet_raw_data/Task002_DRAC/imagesTr'
label_root = './nnUNet_raw_data_base/nnUNet_raw_data/Task002_DRAC/labelsTr'
prepare_data_train(imgs, masks, root, label_root)

imgs = './Test/'
root = './nnUNet_raw_data_base/nnUNet_raw_data/Task002_DRAC/imagesTs'
prepare_data_test(imgs, root)

get_json("Task002_DRAC")

In [ ]:
!nnUNet_plan_and_preprocess -t 002 --verify_dataset_integrity

## Normal training with CV

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2_DA5 Task002_DRAC 0 --npz

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2_DA5 Task002_DRAC 1 --npz

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2_DA5 Task002_DRAC 2 --npz

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2_DA5 Task002_DRAC 3 --npz

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2_DA5 Task002_DRAC 4 --npz

In [ ]:
!nnUNet_find_best_configuration -tr nnUNetTrainerV2_DA5 -m 2d -t 002 

In [ ]:
!nnUNet_predict -i nnUNet_raw_data_base/nnUNet_raw_data/Task002_DRAC/imagesTs/ -o ./output_task2/ -tr nnUNetTrainerV2_DA5 -ctr nnUNetTrainerV2CascadeFullRes -m 2d -p nnUNetPlansv2.1 -t Task002_DRAC

## Task3

In [ ]:
imgs = './Train/Inputs/Task3/img/'
masks = './Train/Inputs/Task3/gt/'
root = './nnUNet_raw_data_base/nnUNet_raw_data/Task003_DRAC/imagesTr'
label_root = './nnUNet_raw_data_base/nnUNet_raw_data/Task003_DRAC/labelsTr'
prepare_data_train(imgs, masks, root, label_root)

imgs = './Test/'
root = './nnUNet_raw_data_base/nnUNet_raw_data/Task003_DRAC/imagesTs'
prepare_data_test(imgs, root)

get_json("Task003_DRAC")

In [ ]:
!nnUNet_plan_and_preprocess -t 003 --verify_dataset_integrity

## Normal training with CV

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2_DA5 Task003_DRAC 0 --npz

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2_DA5 Task003_DRAC 1 --npz

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2_DA5 Task003_DRAC 2 --npz

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2_DA5 Task003_DRAC 3 --npz

In [ ]:
!nnUNet_train 2d nnUNetTrainerV2_DA5 Task003_DRAC 4 --npz

In [ ]:
!nnUNet_find_best_configuration -tr nnUNetTrainerV2_DA5 -m 2d -t 003 

In [ ]:
!nnUNet_predict -i nnUNet_raw_data_base/nnUNet_raw_data/Task003_DRAC/imagesTs/ -o ./output_task3/ -tr nnUNetTrainerV2_DA5 -ctr nnUNetTrainerV2CascadeFullRes -m 2d -p nnUNetPlansv2.1 -t Task003_DRAC

## Postprocessing

In [ ]:
THRESHOLD = 3

for i in range(nr_img):
    # This is just an example of how many images can be used. The amount of model / predictions for each task
    # was slightly different. But the process was always identical
    img1 = cv2.imread(f"./path-to-folder/{i}.png")[:,:,0]
    img2 = cv2.imread(f"./ipath-to-folder/{i}.png")[:,:,0]
    img3 = cv2.imread(f"./path-to-folder/{i}.png")[:,:,0]
    
    img = cv2.merge((img1, img2, img3))
    img = np.sum((img), axis=2).astype(np.uint8)
    img[img>=THRESHOLD] = 1
    img[img!=1] = 0
    img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    img = cv2.flip(img, 1)
    cv2.imwrite(f"./path-to-folder/{i}.png",img)

In [ ]:
THRESHOLD = 3

for i in range(nr_img):
    # This is just an example of how many images can be used. The amount of model / predictions for each task
    # was slightly different. But the process was always identical
    img1 = cv2.imread(f"./path-to-folder/{i}.png")[:,:,0]
    img2 = cv2.imread(f"./path-to-folder/{i}.png")[:,:,0]
    img3 = cv2.imread(f"./path-to-folder/{i}.png")[:,:,0]
    
    img = cv2.merge((img1, img2, img3))
    img = np.sum((img), axis=2).astype(np.uint8)
    img[img>=THRESHOLD] = 1
    img[img!=1] = 0
        
    img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    img = cv2.flip(img, 1)
    cv2.imwrite(f"./images/voting_task2/{i}.png",img)

In [ ]:
for i in range(nr_img):
    # This is just an example of how many images can be used. The amount of model / predictions for each task
    # was slightly different. But the process was always identical
    img1 = cv2.imread(f"./path-to-folder/{i}.png")[:,:,0]
    img2 = cv2.imread(f"./path-to-folder/{i}.png")[:,:,0]
    img3 = cv2.imread(f"./path-to-folder/{i}.png")[:,:,0]
    
    img = cv2.merge((img1, img2, img3))
    img = np.sum((img), axis=2).astype(np.uint8)
    img[img>=1] = 1
    img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    img = cv2.flip(img, 1)
    cv2.imwrite(f"./path-to-folder/{i}.png",img)

In [ ]:
import os
import cv2
import SimpleITK
import numpy as np


def read_nii(nii_path, data_type=np.uint16):
    img = SimpleITK.ReadImage(nii_path)
    data = SimpleITK.GetArrayFromImage(img)
    return np.array(data, dtype=data_type)


def arr2nii(data, filename, reference_name=None):
    img = SimpleITK.GetImageFromArray(data)
    if (reference_name is not None):
        img_ref = SimpleITK.ReadImage(reference_name)
        img.CopyInformation(img_ref)
    SimpleITK.WriteImage(img, filename)


def masks2nii(mask_path, task):
    mask_name_list = os.listdir(mask_path)
    mask_name_list = sorted(mask_name_list, reverse=False, key=lambda x: int(x[:-4]))
    print(mask_name_list)
    mask_list = []
    for mask_name in mask_name_list:
        mask = cv2.imread(os.path.join(mask_path, mask_name), -1)
        mask_list.append(mask)
    arr = np.array(mask_list, np.uint8)
    print(arr.shape)
    arr2nii(arr, f"./path-to-folder/{task}.nii.gz")

In [ ]:
path = f"path-to-folder"
masks2nii(path, 2)